<a href="https://colab.research.google.com/github/saralstalin/FAISS/blob/main/VectorProductSearchWithFAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!pip install -qU langchain-community langchain_openai faiss-gpu gradio

In [76]:
import faiss, csv
import gradio as gr
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS as FaissVectorStore
from google.colab import userdata
from uuid import uuid4
from langchain_core.documents import Document

In [77]:
#need private API Key to use Open AI embeddings
openAIAPIKey = userdata.get('OpenAIAPIKey')
OpenAITextembeddings = OpenAIEmbeddings(model="text-embedding-3-small"
                                        , api_key=openAIAPIKey)

In [78]:
index = faiss.IndexFlatL2(len(OpenAITextembeddings.embed_query("")))

#define vector store
vector_store = FaissVectorStore(
    embedding_function=OpenAITextembeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
#read from products.csv, ignore conversion issues
with open('/content/Products.csv', 'r') as f:
    reader = csv.reader(f)
    documents = []
    for row in reader:
        try:
          price = float(row[2])
          product = Document(page_content=row[0]
                             , metadata={"Quantity": row[1]
                                         , "Price": price})
          documents.append(product)
        except ValueError:
          print(f"Error converting price: {row[2]}")

vector_store.add_documents(documents)



In [80]:
def FindProducts(SearchTerm
                 , PriceMin = 0
                 , PriceMax = 999999
                 , NumberOfResults = 10
                 , MaxSimilarityScore = 2):

    results = vector_store.similarity_search_with_score(SearchTerm
                                                        , k=NumberOfResults,)

    #filter results by price and similarity score
    #for some reason, direct filter in similarity search is not working well.
    filteredResults = [res for res, score in results
                        if res.metadata["Price"] >= PriceMin
                           and res.metadata["Price"] <= PriceMax
                           and score <= MaxSimilarityScore]

    if len(filteredResults) == 0:
        return "No results found"

    #Adding Markdown to show results in a table
    response = "|Product|Quantity|Price|\n"
    response += "|---|---|---|\n"

    #generate a result array with page_content and price alone
    for res in filteredResults:
       response += "|" + res.page_content + "|" + res.metadata["Quantity"] + "|" + str(res.metadata["Price"] ) + "|\n"

    return response



In [ ]:
#testing search result
searchResult = FindProducts("apple", PriceMax=200, MaxSimilarityScore=2)

print(searchResult)

In [ ]:
#building a UI interface for easy testing
iface = gr.Interface(
    fn=FindProducts,
    inputs=[
        gr.Textbox(label="Search", placeholder="Enter a search term"),
        gr.Number(label="Price Min", value=0, ),
        gr.Number(label="Price Max", value=999999),
        gr.Number(label="Max Number of Results", value=10),
        gr.Number(label="Similarity Score", value=1.5),
    ],
    outputs=gr.Markdown(),
    title="Product Search",
    submit_btn="Search",
    clear_btn=None,
)

# Launch the interface
iface.launch()